In [1]:
#Import the necessary libraries
import core.coreapi as coreapi
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib notebook
import plotly.graph_objs as go
from matplotlib import colormaps
import datetime

In [2]:
#Instatiate the API class

qmapi = coreapi.QuantmatixAPI()

Working in environment: PROD


<h1> US POLITICIAN TRADES </h1>

In [3]:
#Define our SQL Query

sql_query = ''' 
SELECT [TickerId]
      ,[FirstName]
      ,[LastName]
      ,[Office]
      ,[Link]
      ,[DateReceived]
      ,[TransactionDate]
      ,[Owner]
      ,[Type]
      ,[Amount]
      ,[Comment]
  FROM [dbo].[TickerSenateTradingFMP]
'''
sql_db_name = 'STUDY_DATABASE_NAME'

In [4]:
#Read in the data
polit_data = qmapi.get_data_sql(sql_string=sql_query,
                                  db_name=sql_db_name)
print(np.shape(polit_data))
polit_data.head()

(9966, 11)


,TickerId,FirstName,LastName,Office,Link,DateReceived,TransactionDate,Owner,Type,Amount,Comment
0,1229,James,Inhofe,James Inhofe,https://efdsearch.senate.gov/search/view/ptr/7...,2015-09-10,2015-08-24,Self,Purchase,"$15,001 - $50,000",None
1,1229,James,Inhofe,James Inhofe,https://efdsearch.senate.gov/search/view/ptr/d...,2016-10-20,2016-09-13,Self,Sale (Full),"$50,001 - $100,000",None
2,1229,Sheldon,Whitehouse,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/5...,2015-08-11,2014-10-02,Self,Purchase,"$1,001 - $15,000",None
3,1229,Sheldon,Whitehouse,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/b...,2015-03-16,2015-02-24,Self,Sale (Full),"$1,001 - $15,000",None
4,1229,Sheldon,Whitehouse,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/b...,2015-03-17,2015-02-24,Self,Sale (Full),"$1,001 - $15,000",None


In [5]:
#first find the types of transactions in the dataset
trans_type = polit_data['Type'].unique()
print(trans_type)

#we have 6 different transaction types

['Purchase' 'Sale (Full)' 'Sale (Partial)' 'Sale' 'Exchange' 'receive']


In [7]:
#this returns the stocks with the most politician trades within our index
traded_comps = polit_data['TickerId'].value_counts().head(50)
traded_comps

TickerId
1241     282
3303     213
3584     134
4573     126
2816     115
12314    112
1528     111
4195     110
2011     108
3701     108
3708     104
3472     104
3407      99
1414      97
2481      95
1949      92
1951      91
2933      87
3589      84
1917      84
4641      83
2522      81
4545      79
1717      76
3283      76
1654      72
1973      72
3906      70
2313      68
2923      66
4590      64
1955      64
2998      64
2737      59
3580      57
1523      57
1244      56
13433     55
1398      55
1887      54
2544      53
1841      53
4465      52
3546      51
1834      48
4415      48
2559      48
2587      47
2542      46
3270      46
Name: count, dtype: int64

In [8]:
ticker_of_choice = 1414


#######################################################################################################

#change dataset to have only the ticker of choice trades
ticker_df = polit_data[(polit_data['TickerId'] == ticker_of_choice)]

#get data of ticker from API
ticker_of_choice_data = qmapi.get_tickers_data(tickers_list=[ticker_of_choice])

#our start date is the first recorded trade for reading in Yahoo data
start_date=ticker_df['TransactionDate'].min()

#our end date is today
end_date=datetime.date.today()

#ticker short name got from API data
ticker = ticker_of_choice_data['shortName'].str.split(':').str[0]

#read in historical price data
ticker_data = yf.download(ticker[0], start=start_date, end=end_date, interval='1d')


########################################################################################################

#CREATE THE PLOT

highlight_dates  = ticker_df['TransactionDate'] #take dates of transactions
highlight_dates = pd.to_datetime(highlight_dates) #convert to datetime
transaction_type = ticker_df['Type'] #take all transaction types associated with the dates

highlight_df = pd.DataFrame({'Date':highlight_dates, 'Type':transaction_type}) #make a new dataframe containing dates and types

#find the number of unique transaction types
unique_types = highlight_df['Type'].unique()
num_types = len(unique_types)

# Use a colormap to assign colors (using matplotlib for color generation)
colors = colormaps['Set1'](np.linspace(0, 1, num_types))  
color_map = {typ: f'rgb({int(c[0] * 255)}, {int(c[1] * 255)}, {int(c[2] * 255)})'
              for typ, c in zip(unique_types, colors)}  # Map each type to a color

def get_next_available_date(date, ticker_index):
    max_date = ticker_index.max()
    while date not in ticker_index and date <= max_date:
        date += pd.Timedelta(days=1)
    return date if date in ticker_index else None

#plot ticker and transaction dates
fig = go.Figure()

#plot ticker data
fig.add_trace(go.Scatter(x=ticker_data.index, 
                         y=ticker_data['Close'],
                         mode='lines',
                         name=f'{ticker[0]} Daily Close Price'))

#plot transaction dates and types
legend_added = set()

for _, row in highlight_df.iterrows():
    date = row['Date']
    type = row['Type']
    adjusted_date = date if date in ticker_data.index else get_next_available_date(date, ticker_data.index)
    if adjusted_date in ticker_data.index:
        fig.add_trace(go.Scatter(x = [date],
                                y = [ticker_data.loc[adjusted_date, 'Close']],
                                mode='markers',
                                name=type,
                                marker=dict(color=color_map[type], size=8),
                                showlegend=type not in legend_added,
                                legendgroup=type,
                                visible='legendonly'
                                ))
        legend_added.add(type)
    else:
        print(f"Date {date} not found in ticker_data index.") #sometimes the date is not in the ticker data, so this avoids an error
    

fig.update_layout(
    title = f'{ticker_of_choice_data['instrumentName'].iloc[0]} Historical Price & US Politician Trades',
    xaxis_title = 'Date',
    yaxis_title = 'Price',
    showlegend = True,
    legend = dict(groupclick="togglegroup")
)

fig.show()

[*********************100%%**********************]  1 of 1 completed
